In [1]:
import matplotlib
import torchmetrics
%matplotlib inline

c:\Users\User\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Translation with a Sequence to Sequence Network
*************************************************************

::

    [KEY: 
    > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

... to varying degrees of success.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  http://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__

You will also find the previous tutorials on
:doc:`/intermediate/char_rnn_classification_tutorial`
and :doc:`/intermediate/char_rnn_generation_tutorial`
helpful as those concepts are very similar to the Encoder and Decoder
models, respectively.

And for more, read the papers that introduced these topics:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__


**Requirements**



Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site http://tatoeba.org/ which has
downloads available at http://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.



In [2]:
#Only for Google Colab
# !wget http://www.manythings.org/anki/fra-eng.zip
# !unzip -o fra-eng.zip
# !mkdir data
# !mv fra.txt data/eng-fra.txt

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)   #separates punctuations from surroundingg words by a space
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)   #removes any characters except letters, punctuation marks, and spaces.
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')  #%s-%s.txt replaces the first %s with lang1 and second %s with lang2

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]        #Split using \t as the delimiter, which stands for tab. [:2] means the two languages

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [6]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 232736 sentence pairs
Trimmed to 22907 sentence pairs
Counting words...
Counted words:
fra 7019
eng 4638
['je suis de ton cote .', 'i m by your side .']


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#Input first language, target second language
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
train_pairs = list(zip(X_train,y_train))
test_pairs = list(zip(X_test,y_test))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.






In [12]:
#Positional encoding for Transformer encoder
import math
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [13]:
class EncoderRNN(nn.Module):    #GRU
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size) #Gated Recurrent Unit

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class EncoderLSTM(nn.Module):   #LSTM
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size) 
        

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, (hidden,cell) = self.lstm(output, (hidden, cell))
        return output, hidden, cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    
class EncoderbiLSTM(nn.Module): #biLSTM
    def __init__(self, input_size, hidden_size):
        super(EncoderbiLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.bilstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True) 
        

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, (hidden,cell) = self.bilstm(output, (hidden, cell))
        return output, hidden, cell

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)
    

class EncoderTransformer(nn.Module):    #Transformer
    def __init__(self, input_size, hidden_size, MAX_LENGTH):
        super(EncoderTransformer, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size, 0, MAX_LENGTH)

        self.encoder_layer = nn.TransformerEncoderLayer(hidden_size, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers = 6) 

    def forward(self, input):
        embedded = self.embedding(input).view(-1, 1, self.hidden_size)
        output = self.pos_encoder(embedded)
        output = self.transformer_encoder(output)
        return output


The Decoder (Your assignment)
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).


In [14]:
class Decoder(nn.Module):   #GRU: specify if we want to use attention mechanism or not.
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.out_attn = nn.Linear(hidden_size*2, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs = None, attention = False):

        # Your code here #
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        if attention:
            # print(hidden.size(), encoder_outputs.size())
            attention_scores = torch.einsum('bij, kj->bik', hidden, encoder_outputs)    #hidden = [batch, L, hidden_dim], encoder_output=[L, hidden_dim]
            attention_weights = F.softmax(attention_scores, dim=2)  #attention_weights = [batch, L, L]
            context = torch.einsum('bik,kj->bij', attention_weights, encoder_outputs)
            output = torch.cat((context, output), dim=2)
            output = self.softmax(self.out_attn(output[0]))

        else:
            output = self.softmax(self.out(output[0]))

        return output, hidden
    

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderLSTM(nn.Module):   #LSTM
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):

        # Your code here #
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, (hidden,cell) = self.lstm(output, (hidden, cell))
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
#Same as RNN but change input hidden dim to hidden_size*2
class DecoderforbiLSTM(nn.Module):  #GRU for biLSTM encoder
    def __init__(self, hidden_size, output_size):
        super(DecoderforbiLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)       
        self.out = nn.Linear(hidden_size, output_size)      
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # Your code here #
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)



<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [15]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [16]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, attention: bool, max_length=MAX_LENGTH):

    '''For RNN and LSTM'''
    # encoder_hidden = encoder.initHidden()

    '''For LSTM and biLSTM'''
    # encoder_cell = encoder.initHidden() 

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    '''For RNN and LSTM and Transformer'''
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    ''''For biLSTM'''
    # encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

    loss = 0

    '''For RNN'''
    # for ei in range(input_length):
    #     encoder_output, encoder_hidden = encoder(
    #         input_tensor[ei], encoder_hidden)
    #     encoder_outputs[ei] = encoder_output[0, 0]  #output at time step 0, hidden dim 0


    '''For LSTM and biLSTM'''
    # for ei in range(input_length):
    #   encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
    #   encoder_outputs[ei] = encoder_output[0, 0]  
 
    '''For Transformer'''
    encoder_outputs = encoder(input_tensor).squeeze(1)
    encoder_hidden = torch.mean(encoder_outputs, dim=0, keepdim=True)
  

 
    decoder_input = torch.tensor([[SOS_token]], device=device)

    '''For RNN, LSTM, biLSTM and Transformer (with attention at decoder)'''
    decoder_hidden = encoder_hidden.view(1,1,-1)

    '''For LSTM and biLSTM'''
    # decoder_cell = encoder_cell.view(1,1,-1)
    


    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input

        '''For RNN'''
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, attention = attention, encoder_outputs = encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

        '''For LSTM'''
        # for di in range(target_length):
        #     decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, decoder_cell)
        #     loss += criterion(decoder_output, target_tensor[di])
        #     decoder_input = target_tensor[di]  

        '''For biLSTM'''
        # for di in range(target_length):
        #     decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        #     loss += criterion(decoder_output, target_tensor[di])
        #     decoder_input = target_tensor[di]  # Teacher forcing



    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            '''For RNN'''
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, attention = attention, encoder_outputs=encoder_outputs)

            '''For LSTM'''
            # decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, decoder_cell)
            
            '''For biLSTM'''
            # decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [17]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [18]:
def trainIters(encoder, decoder, epochs, attention, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print("Epoch: %d/%d" % (epoch, epochs))
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion, attention)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            iter +=1

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [28]:
def evaluate(encoder, decoder, sentence, attention, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        # encoder_hidden = encoder.initHidden()

        '''For LSTM and biLSTM'''
        # encoder_cell = encoder.initHidden() 

        '''For RNN and LSTM and Transformer'''
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        '''For biLSTM'''
        # encoder_outputs = torch.zeros(max_length, 2*encoder.hidden_size, device=device)

        '''For RNN'''
        # for ei in range(input_length):
        #     encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        #     encoder_outputs[ei] += encoder_output[0, 0]

        '''For biLSTM and LSTM'''
        # for ei in range(input_length):
        #     encoder_output, encoder_hidden, encoder_cell= encoder(input_tensor[ei], encoder_hidden, encoder_cell)
        #     encoder_outputs[ei] += encoder_output[0, 0]


        '''For Transformer'''
        encoder_outputs = encoder(input_tensor).squeeze(1)
        encoder_hidden = torch.mean(encoder_outputs, dim=0, keepdim=True)
        
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        '''For RNN, LSTM, biLSTM, Transformer'''
        decoder_hidden = encoder_hidden.view(1,1,-1)
        '''For LSTM and biLSTM'''
        # decoder_cell = encoder_cell.view(1,1,-1) 

        decoded_words = []

        for di in range(max_length):
            '''For RNN'''
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, attention = attention, encoder_outputs = encoder_outputs)
            
            '''For LSTM'''
            # decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, decoder_cell)

            '''For biLSTM'''
            # decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [20]:
def evaluateRandomly(encoder, decoder, attention, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], attention)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [21]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm
import numpy as np

rouge = ROUGEScore()

def test(encoder, decoder, testing_pairs, attention):
    input = []
    gt = []
    predict = []
    metric_score = {
        "rouge1_fmeasure":[],
        "rouge1_precision":[],
        "rouge1_recall":[],
        "rouge2_fmeasure":[],
        "rouge2_precision":[],
        "rouge2_recall":[]
    }
    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate(encoder, decoder, pair[0], attention = attention)
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

        try:
            rs = rouge(output_sentence, pair[1])
        except:
            continue
        metric_score["rouge1_fmeasure"].append(rs['rouge1_fmeasure'])
        metric_score["rouge1_precision"].append(rs['rouge1_precision'])
        metric_score["rouge1_recall"].append(rs['rouge1_recall'])
        metric_score["rouge2_fmeasure"].append(rs['rouge2_fmeasure'])
        metric_score["rouge2_precision"].append(rs['rouge2_precision'])
        metric_score["rouge2_recall"].append(rs['rouge2_recall'])

    metric_score["rouge1_fmeasure"] = np.array(metric_score["rouge1_fmeasure"]).mean()
    metric_score["rouge1_precision"] = np.array(metric_score["rouge1_precision"]).mean()
    metric_score["rouge1_recall"] = np.array(metric_score["rouge1_recall"]).mean()
    metric_score["rouge2_fmeasure"] = np.array(metric_score["rouge2_fmeasure"]).mean()
    metric_score["rouge2_precision"] = np.array(metric_score["rouge2_precision"]).mean()
    metric_score["rouge2_recall"] = np.array(metric_score["rouge2_recall"]).mean()

    print("=== Evaluation score - Rouge score ===")
    print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"])
    print("Rouge1 precision:\t",metric_score["rouge1_precision"])
    print("Rouge1 recall:  \t",metric_score["rouge1_recall"])
    print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"])
    print("Rouge2 precision:\t",metric_score["rouge2_precision"])
    print("Rouge2 recall:  \t",metric_score["rouge2_recall"])
    print("=====================================")
    return input,gt,predict,metric_score

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [22]:
hidden_size = 512

'''Encoder1, decoder1: RNN '''
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)
# trainIters(encoder1, decoder1, 5, print_every=5000)

'''Encoder2, decoder2: LSTM'''
# encoder2 = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
# decoder2 = DecoderLSTM(hidden_size, output_lang.n_words).to(device)
# trainIters(encoder2, decoder2, 5, print_every=5000)

'''Encoder3: biLSTM, Decoder3: RNN'''
# encoder3 = EncoderbiLSTM(input_lang.n_words, hidden_size).to(device)
# #decoder3 =  DecoderforbiLSTM(hidden_size*2, output_lang.n_words).to(device) #Twice hidden dim because of concat of hidden states
# decoder3 = DecoderLSTM(hidden_size*2, output_lang.n_words).to(device)   #(Expt)LSTM decoder
# trainIters(encoder3, decoder3, 5, print_every=5000)

'''Encoder4, decoder4: RNN with attention'''
# encoder4 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder4 = Decoder(hidden_size, output_lang.n_words).to(device)
# trainIters(encoder4, decoder4, 5, attention = True,print_every=5000)

'''Encoder5: Transformer, Decoder5: RNN'''
encoder5 = EncoderTransformer(input_lang.n_words, hidden_size, MAX_LENGTH).to(device)
decoder5 = Decoder(hidden_size, output_lang.n_words).to(device)
trainIters(encoder5, decoder5,  5, attention=True, print_every=5000)


c:\Users\User\anaconda3\envs\nlp\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\User\anaconda3\envs\nlp\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: 0/5
1m 11s (- 23m 18s) (5000 4%) 3.4065
2m 25s (- 22m 30s) (10000 9%) 2.9481
3m 38s (- 21m 24s) (15000 14%) 2.8259
4m 53s (- 20m 17s) (20000 19%) 2.7269
Epoch: 1/5
6m 7s (- 19m 8s) (25000 24%) 2.6099
7m 22s (- 17m 57s) (30000 29%) 2.5037
8m 36s (- 16m 44s) (35000 33%) 2.4208
9m 50s (- 15m 31s) (40000 38%) 2.3879
Epoch: 2/5
11m 5s (- 14m 19s) (45000 43%) 2.3242
12m 21s (- 13m 7s) (50000 48%) 2.2635
13m 35s (- 11m 52s) (55000 53%) 2.2075
14m 49s (- 10m 38s) (60000 58%) 2.2100
Epoch: 3/5
16m 3s (- 9m 24s) (65000 63%) 2.1513
17m 17s (- 8m 10s) (70000 67%) 2.1539
18m 31s (- 6m 56s) (75000 72%) 2.1447
19m 45s (- 5m 41s) (80000 77%) 2.1327
Epoch: 4/5
20m 58s (- 4m 27s) (85000 82%) 2.1526
22m 13s (- 3m 13s) (90000 87%) 2.1551
23m 27s (- 1m 59s) (95000 92%) 2.1302
24m 40s (- 0m 45s) (100000 97%) 2.1700


In [29]:
#evaluateRandomly(encoder1, decoder1)
#evaluateRandomly(encoder2, decoder2)
# evaluateRandomly(encoder3, decoder3)
# evaluateRandomly(encoder4, decoder4)
evaluateRandomly(encoder5, decoder5, attention = True)

> tu es vire .
= you re fired .
< you re stuck . <EOS>

> c est un scientifique .
= he is a scientist .
< he is a man . . <EOS>

> j attends l homme ideal .
= i m waiting for the right man .
< i m the to the . <EOS>

> elle passe un peu de temps chaque jour a lire la bible .
= she spends a little time each day reading the bible .
< i m a little tired of <EOS>

> elle arreta de parler .
= she stopped talking .
< she s about the <EOS>

> je suis certain que vous ne me decevrez pas .
= i m sure you won t disappoint me .
< i m glad you re you . . <EOS>

> nous restons .
= we re staying .
< we re going . <EOS>

> je suis plus elegante que vous .
= i m smarter than you .
< i m smarter than you . <EOS>

> je vous rends nerveuses n est ce pas ?
= i make you nervous don t i ?
< i m really not worried about you ? <EOS>

> je suis desole si je t ai effraye .
= i m sorry if i frightened you .
< i m sorry if i embarrassed you <EOS>



In [30]:
#ROUGUE for train pairs
#input,gt,predict,score = test(encoder1, decoder1, train_pairs)
#input,gt,predict,score = test(encoder2, decoder2, train_pairs)
# input,gt,predict,score = test(encoder3, decoder3, train_pairs)
#input,gt,predict,score = test(encoder4, decoder4, train_pairs)
input,gt,predict,score = test(encoder5, decoder5, train_pairs, attention = True)

100%|██████████| 20616/20616 [02:43<00:00, 125.98it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.4772951
Rouge1 precision:	 0.46914554
Rouge1 recall:  	 0.49953288
Rouge2 fmeasure:	 0.28600445
Rouge2 precision:	 0.2779501
Rouge2 recall:  	 0.30644673


In [31]:
#ROGUE for test pairs
# input,gt,predict,score = test(encoder1, decoder1, test_pairs)
# input,gt,predict,score = test(encoder2, decoder2, test_pairs)
# input,gt,predict,score = test(encoder3, decoder3, test_pairs)
# input,gt,predict,score = test(encoder4, decoder4, test_pairs)
input,gt,predict,score = test(encoder5, decoder5, test_pairs, attention = True)

100%|██████████| 2291/2291 [00:18<00:00, 125.48it/s]

=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.46973753
Rouge1 precision:	 0.4596179
Rouge1 recall:  	 0.49348828
Rouge2 fmeasure:	 0.28015462
Rouge2 precision:	 0.27129158
Rouge2 recall:  	 0.30109358
